In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import faiss

load_dotenv()

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

PARQUET_PATH = os.getenv("PARQUET_PATH")
assert PARQUET_PATH is not None, "PARQUET_PATH variable is missing!"

In [2]:
df = pd.read_parquet(PARQUET_PATH)
df.head()

,doc_id,MeghozoBirosag,JogTerulet,Jogszabalyhelyek,HatarozatEve,AllKapcsolodoUgyszam,AllKapcsolodoBirosag,KapcsolodoHatarozatok,text,embedding
0,Bf.262/2011/12,Pécsi Ítélőtábla,büntetőjog,1998. évi XIX. törvény a büntetőeljárásról 348...,2011.0,"[""B.54/2010/26""]","[""Szekszárdi Törvényszék""]","[{""KapcsolodoUgyszam"": ""B.54/2010/26"", ""Kapcso...","A Pécsi Ítélőtábla, mint másodfokú bíróság Bf....","[-0.025117767974734306, 0.037775538861751556, ..."
1,Bf.34/2015/29,Pécsi Ítélőtábla,büntetőjog,1998. évi XIX. törvény a büntetőeljárásról 335...,2016.0,"[""B.11/2014/18""]","[""Zalaegerszegi Törvényszék""]","[{""KapcsolodoUgyszam"": ""B.11/2014/18"", ""Kapcso...","A Pécsi Ítélőtábla, mint másodfokú bíróság Bf....","[-0.028827665373682976, 0.03442743048071861, -..."
2,Bf.111/2007/8,Pécsi Ítélőtábla,büntetőjog,1998. évi XIX. törvény a büntetőeljárásról 351...,2007.0,"[""Bhar.966/2007/10"", ""B.185/2006/18""]","[""Kúria"", ""Szekszárdi Törvényszék""]","[{""KapcsolodoUgyszam"": ""Bhar.966/2007/10"", ""Ka...","A Pécsi Ítélőtábla, mint másodfokú bíróság Bf....","[-0.031228363513946533, 0.01903749816119671, -..."
3,Bf.95/2014/10,Pécsi Ítélőtábla,büntetőjog,1978. évi IV. törvény a Büntető Törvénykönyvrő...,2015.0,"[""B.226/2012/189""]","[""Pécsi Törvényszék""]","[{""KapcsolodoUgyszam"": ""B.226/2012/189"", ""Kapc...","A Pécsi Ítélőtábla, mint másodfokú bíróság Bf....","[-0.02428455650806427, 0.018554724752902985, 0..."
4,Bhar.181/2012/8,Pécsi Ítélőtábla,büntetőjog,1998. évi XIX. törvény a büntetőeljárásról 387...,2012.0,"[""Bf.1/2012/3"", ""B.151/2011/8""]","[""Szekszárdi Törvényszék"", ""Tamási Járásbíróság""]","[{""KapcsolodoUgyszam"": ""Bf.1/2012/3"", ""Kapcsol...","A Pécsi Ítélőtábla, mint harmadfokú bíróság Bh...","[0.002832422498613596, 0.015901876613497734, -..."


In [3]:
sample = df["embedding"].iloc[0]
print("Embedding dimension:", len(sample))

Embedding dimension: 1536


In [5]:
# Adattisztítás az index építése előtt
initial_rows = len(df)
df_cleaned = df.dropna(subset=['embedding']).copy()

# A beágyazások dimenzióinak ellenőrzése
EXPECTED_DIM = 1536
mask = df_cleaned['embedding'].apply(lambda x: isinstance(x, np.ndarray) and x.shape[0] == EXPECTED_DIM)
df_cleaned = df_cleaned[mask]

removed_rows = initial_rows - len(df_cleaned)
print(f"Eredeti sorok száma: {initial_rows}")
print(f"Tisztítás utáni sorok száma: {len(df_cleaned)}")
print(f"Eltávolított sorok száma: {removed_rows}")

# Index építése a tisztított adatokból
print("\nFAISS index építése...")
embedding_matrix = np.vstack(df_cleaned["embedding"].values)
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)
print("FAISS index sikeresen felépítve.")

# Fontos: Az ID mappinget is a tisztított DataFrame-ből kell készíteni!
# A df.iterrows() helyett a df_cleaned.reset_index().iterrows() használata biztosítja a helyes megfeleltetést.
id_mapping = {i: row["doc_id"] for i, row in df_cleaned.reset_index(drop=True).iterrows()}

Eredeti sorok száma: 213398
Tisztítás utáni sorok száma: 213351
Eltávolított sorok száma: 47

FAISS index építése...
FAISS index sikeresen felépítve.


In [6]:
faiss.write_index(index, "data/processed/faiss_index.bin")

id_mapping = {i: row["doc_id"] for i, row in df.iterrows()}
import pickle
with open("data/processed/faiss_id_mapping.pkl", "wb") as f:
    pickle.dump(id_mapping, f)

In [7]:
faiss.write_index(index, "data/processed/faiss_index.bin")

id_mapping = {i: row["doc_id"] for i, row in df.iterrows()}
import pickle
with open("data/processed/faiss_id_mapping.pkl", "wb") as f:
    pickle.dump(id_mapping, f)

In [8]:
query = df["embedding"].iloc[0]
_, indices = index.search(np.array([query]), k=5)
[ id_mapping[i] for i in indices[0] ]

['Bf.262/2011/12',
 'Bf.71/2023/39',
 'Bf.255/2007/9',
 'Bhar.38/2016/13',
 'Bf.273/2011/17']